#### Set Delta V-Order and Delta Configs

In [9]:
spark.conf.set("spark.sql.parquet.vorder.enabled","true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enable","true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize","1073741824")

StatementMeta(, 4c616c88-a843-4231-a116-5c9f2cfbe1bc, 11, Finished, Available)

#### Load Dates from Shortcut Bronze table, enrich with columns, save to Silver table

In [8]:
dfBronzeDates = spark.read.format("delta").load("Tables/bronze_dates")

StatementMeta(, 4c616c88-a843-4231-a116-5c9f2cfbe1bc, 10, Finished, Available)

In [11]:
from pyspark.sql.functions import concat,concat_ws,col,lpad,substring,lit,to_date

dfBronzeDates \
    .select("*", \
        concat("Year",lpad("Month",2,"0")).alias("MonthOfYearNumber"), \
        concat("Year",lpad("Quarter",2,"0")).alias("QuarterOfYearNumber"), \
        concat_ws(" ",substring("MonthName",1,3),"Year").alias("MonthOfYear"),\
        concat(lit("Q"),"Quarter",lit(" "),"Year").alias("QuarterOfYear")) \
    .write.mode("overwrite").format("delta").save("Tables/" + "silver_dates")

StatementMeta(, 4c616c88-a843-4231-a116-5c9f2cfbe1bc, 13, Finished, Available)

#### Load Shortcut Bronze NYC Yellow Taxi data to Dataframe

In [38]:
dfBronzeTaxi = spark.read.format("delta").load("Tables/bronze_nyctlcYellow")

StatementMeta(, 4c616c88-a843-4231-a116-5c9f2cfbe1bc, 40, Finished, Available)

In [40]:
#from pyspark.sql.functions import to_date

from pyspark.sql.functions import *

dfBronzeTaxi \
    .select("*", \
        to_date(dfBronzeTaxi["tpepPickupDateTime"]).alias("tpep_pickup_date"), \
        to_date(dfBronzeTaxi["tpepDropoffDateTime"]).alias("tpep_dropoff_date")) \
    .write.mode("overwrite").format("delta").save("Tables/silver_nyctlcYellow")


StatementMeta(, 4c616c88-a843-4231-a116-5c9f2cfbe1bc, 42, Finished, Available)

In [41]:
%%sql
OPTIMIZE silver_nyctlcYellow

StatementMeta(, 4c616c88-a843-4231-a116-5c9f2cfbe1bc, 43, Finished, Available)

<Spark SQL result set with 1 rows and 2 fields>